In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
%load_ext tensorboard.notebook

In [ ]:
print("Loading MNIST ...", end = " ");
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
print("done.")

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

print(model.summary())

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')])

print(model.summary())

Functional interface:

In [ ]:
inputs = tf.keras.Input(shape=(28, 28, 1))
l = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))(inputs)
l = tf.keras.layers.MaxPooling2D((2, 2))(l)
l = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(l)
l = tf.keras.layers.MaxPooling2D((2, 2))(l)
l = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(l)
l = tf.keras.layers.Flatten()(l)
l = tf.keras.layers.Dense(64, activation='relu')(l)
predictions = tf.keras.layers.Dense(10, activation='softmax')(l)

# Instantiate model
model = tf.keras.Model(inputs=inputs, outputs=predictions)

print(model.summary())

In [ ]:
callbacks = [
  tf.keras.callbacks.TensorBoard(log_dir='./logs', update_freq='batch')
]
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=32, epochs=5, callbacks=callbacks)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test accuracy:", test_acc)

Model subclassing API:

In [ ]:
class MyModel(tf.keras.Model):
 def __init__(self):
   super(MyModel, self).__init__()
   # Define your layers here.
   self.conv_1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))
   self.pool_1 = tf.keras.layers.MaxPooling2D((2, 2))
   self.conv_2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')
   self.pool_2 = tf.keras.layers.MaxPooling2D((2, 2))
   self.conv_3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')
   self.flat = tf.keras.layers.Flatten()
   self.dense = tf.keras.layers.Dense(64, activation='relu')
   self.predictions = tf.keras.layers.Dense(10, activation='softmax')
 def call(self, inputs, training=None):
   # Define your forward pass here,
   # using layers you previously defined in `__init__`
   # With the Boolean training flag, you can have different bahavior in training and testing 
   # (e.g., needed for dropout, batch normalization, ...)
   return self.predictions(self.dense(self.flat(self.conv_3(self.pool_2(self.conv_2(self.pool_1(self.conv_1(inputs))))))))

model = MyModel()

callbacks = [
  tf.keras.callbacks.TensorBoard(log_dir='./logs', update_freq='batch')
]
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=32, epochs=5, callbacks=callbacks)

print(model.summary()) # Has to come after model.fit 

test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test accuracy:", test_acc)

In [ ]:
%tensorboard --logdir logs